In [1]:
# from thuglm.modeling_chatglm import ChatGLMForConditionalGeneration
from transformers import AutoConfig, AutoTokenizer, AutoModel
import os 
import torch

In [2]:
model_name_or_path = "D:\\训练代码\\ptuning\\output\\adgen-chatglm-6b-pt-$PRE_SEQ_LEN-$LR\\checkpoint-50"

In [3]:
config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
config.pre_seq_len = 8
config.prefix_projection = False# model_args.prefix_projection

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


In [4]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
model = AutoModel.from_pretrained("THUDM/chatglm-6b", config=config, trust_remote_code=True)
prefix_state_dict = torch.load(os.path.join(model_name_or_path, "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)
model = model.eval()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at THUDM/chatglm-6b and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model = model.half()
model.transformer.prefix_encoder.float()

PrefixEncoder(
  (embedding): Embedding(8, 229376)
)

In [8]:
model = model.cuda()

In [17]:
response, history = model.chat(tokenizer, "你好", history=[])
print(response)

The dtype of attention mask (torch.int64) is not bool


你好，我是ChatGLM。很高兴能和你聊天。
